Apriori is an algorithm for frequent item set mining and association rule learning over relational databases. as you know, this algorithm is often used for market basket analysis. but with a bit of creativity, I used it for another fun purpose. I used Apriori to analyze the emojis that people often used together😎.

so let us begin🤝.

after replacing api credentials from http://my.telegram.org run the extract.py file to get a groups-messages.json file.
this file contains messages from your telegram groups.
if you didnt succeed in running the code, i have provided my extraction in this google drive folder.
https://drive.google.com/drive/folders/1QYGRtv5sRCwfJXYvnBfSsHJDjvWBmZN8?usp=sharing
just download it and you can follow along. keep in mind my groups language was Persian.

as always import numpy and pandas. numpy helps us in the calculation stuff and pandas helps us handle the dataset.

lets load the data and have a sneak look at the type of the columns by this code block.
that would be the first step of our data mining process. ill name it getting to know your data.

In [38]:
import pandas as pd
import numpy as np

messages = pd.read_json("groups-messages-1000.json")

messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41809 entries, 0 to 41808
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   _                   41809 non-null  object             
 1   id                  41809 non-null  int64              
 2   peer_id             41809 non-null  object             
 3   date                41809 non-null  datetime64[ns, UTC]
 4   message             41703 non-null  object             
 5   out                 41809 non-null  bool               
 6   mentioned           41809 non-null  bool               
 7   media_unread        41809 non-null  bool               
 8   silent              41809 non-null  bool               
 9   post                41809 non-null  bool               
 10  from_scheduled      41703 non-null  float64            
 11  legacy              41809 non-null  bool               
 12  edit_hide           41703 non-nu

as you can see the dataset has 31 columns. its obvious that we dont need all of them.

we only need the one that provides us with the body of the message. this would be the message column.
so messages['message'] is the series that we want to analyze.

In [39]:
# prepair dataset for analysis
# remove empty messages
messages['message'].replace('', np.nan, inplace=True)

messages.dropna(subset=['message'], inplace=True)
messages.head()

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,views,forwards,replies,edit_date,post_author,grouped_id,restriction_reason,ttl_period,sender,action
0,Message,3417305,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:21:20+00:00,😂😂,False,False,False,False,False,...,NaN,NaN,None,None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 1068297745, 'is_self': Fal...",NaN
1,Message,3417304,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:21:20+00:00,😂😂🗿,False,False,False,False,False,...,NaN,NaN,None,None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 2055945179, 'is_self': Fal...",NaN
3,Message,3417302,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:21:19+00:00,استیکر به عکس,False,False,False,False,False,...,NaN,NaN,None,None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 1773058448, 'is_self': Fal...",NaN
4,Message,3417301,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:21:09+00:00,پشمام چق پاش بزرگه,False,False,False,False,False,...,NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 5529965814, 'is_self': Fal...",NaN
7,Message,3417298,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:21:02+00:00,تعطیل,False,False,False,False,False,...,NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 5457357528, 'is_self': Fal...",NaN


so basically Apriori takes every customers transaction and based on the items that the customer bought together it creates some rules.

in our situation the items are emojis. so it makes sense to delete messages that doesnt contain any emoji.
we can do this by this code block.

In [40]:
import emoji

messages.drop(messages[messages['message'].str.match(emoji.get_emoji_regexp()) == False ].index, inplace=True)
messages.head()

C:\Users\llabb\AppData\Local\Temp\ipykernel_16124\2074604905.py:3: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  messages.drop(messages[messages['message'].str.match(emoji.get_emoji_regexp()) == False ].index, inplace=True)


,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,views,forwards,replies,edit_date,post_author,grouped_id,restriction_reason,ttl_period,sender,action
0,Message,3417305,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:21:20+00:00,😂😂,False,False,False,False,False,...,NaN,NaN,None,None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 1068297745, 'is_self': Fal...",NaN
1,Message,3417304,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:21:20+00:00,😂😂🗿,False,False,False,False,False,...,NaN,NaN,None,None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 2055945179, 'is_self': Fal...",NaN
45,Message,3417258,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:19:19+00:00,🤪😂,False,False,False,False,False,...,NaN,NaN,None,None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 1068297745, 'is_self': Fal...",NaN
93,Message,3417206,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:17:04+00:00,😐,False,False,False,False,False,...,NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 5004751348, 'is_self': Fal...",NaN
126,Message,3417171,"{'_': 'PeerChannel', 'channel_id': 1188848766}",2022-07-02 19:14:29+00:00,✈️👻,False,False,False,False,False,...,NaN,NaN,None,None,NaN,NaN,[],NaN,"{'_': 'User', 'id': 1068297745, 'is_self': Fal...",NaN


that will leave us a dataset of messages that contain emojis

afterward, we need to change the form of the dataset to the schema that Apriori accepts and understands. ill call this stage the transforming step.

for Apriori, we need to have a column for each item ( product so to say ). to do this run this code:

In [41]:
# create a new operation dataframe to transform the dataset into a new format
emojis = pd.DataFrame(columns=emoji.EMOJI_DATA)
emojis['message'] = messages['message']
emojis2 = emojis.copy()
emojis.head()

,🥇,🥈,🥉,🆎,🏧,🅰️,🅰,🇦🇫,🇦🇱,🇩🇿,...,☯️,☯,🪀,🤪,🦓,🤐,🧟,💤,🇦🇽,message
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,😂😂
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,😂😂🗿
45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,🤪😂
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,😐
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✈️👻


this will give us a dataset with 4704 columns 🤯. a column for each emoji + one for the message itself.

as you can tell all the rows are NaN. we can fill the rows according to the message body by this code block.

In [42]:
for column in emojis :
    if column == 'message':
        continue
    try :
        emojis[column] = emojis.message.str.contains(column)
    except :
        pass

emojis.fillna(False, inplace=True)
emojis.drop(columns=['message'], inplace=True)
emojis.tail()

,🥇,🥈,🥉,🆎,🏧,🅰️,🅰,🇦🇫,🇦🇱,🇩🇿,...,💴,☯️,☯,🪀,🤪,🦓,🤐,🧟,💤,🇦🇽
41759,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41764,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41769,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41796,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41800,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


to mine the rules by Apriori first, we have to extract frequent itemsets by the Apriori algorithm.
thankfully python makes it super easy. well do this by running this code:

In [50]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(emojis, min_support=0.009, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.017324,(👇)
1,0.011870,(💔)
2,0.011870,(✅)
3,0.016041,(🚬)
4,0.014116,(😕)
...,...,...
85,0.009945,"(🏻, 🚶, ♀, 🚶🏻‍♀)"
86,0.009945,"(🏻, 🚶🏻‍♀, ♀, 🚶🏻)"
87,0.009945,"(🚶🏻‍♀, 🚶, ♀, 🚶🏻)"
88,0.009945,"(🏻, 🚶🏻‍♀, 🚶, 🚶🏻)"


finally, we have to provide them to the association rule function by :

In [51]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=10)
rules.sort_values('confidence', ascending=False, inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
103,"(🦯, 👩‍🦯)",(👩),0.009625,0.010908,0.009625,1.00000,91.676471,0.009520,inf
107,(👩‍🦯),"(🦯, 👩)",0.009625,0.010908,0.009625,1.00000,91.676471,0.009520,inf
77,(🚶🏻‍♀),"(🏻, 🚶)",0.009945,0.019891,0.009945,1.00000,50.274194,0.009748,inf
140,"(🏻, ♀, 🚶🏻‍♀)",(🚶🏻),0.009945,0.019891,0.009945,1.00000,50.274194,0.009748,inf
75,"(🚶, 🚶🏻‍♀)",(🏻),0.009945,0.033365,0.009945,1.00000,29.971154,0.009614,inf
...,...,...,...,...,...,...,...,...,...
138,(♀),"(🏻, 🚶, 🚶🏻‍♀)",0.068335,0.009945,0.009945,0.14554,14.633803,0.009266,1.15869
68,(♀),"(🚶🏻, 🚶🏻‍♀)",0.068335,0.009945,0.009945,0.14554,14.633803,0.009266,1.15869
164,(♀),"(🚶, 🚶🏻, 🚶🏻‍♀)",0.068335,0.009945,0.009945,0.14554,14.633803,0.009266,1.15869
62,(♀),"(🚶, 🚶🏻‍♀)",0.068335,0.009945,0.009945,0.14554,14.633803,0.009266,1.15869


with that done we have ower rules

how do you want to choose good rules is up to you.

i chose them by considering lift and support and just observing the rule. The best rules that i did find are the following:

😂=>🤣
👩‍🦯=>🦯
😍=>❤️
😒=>😤
so thats it for this journey. i hope you enjoyed and learned cuz i did.

a star would be awesome😌❤️

